In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 15 11:11:13 2020

@author: 匡查理
"""


def Hscore(df, Team):
    data = df.loc[df['TeamID'] == Team]
    groupOri = data.groupby(['MatchID', 'OriginPlayerID']).count()
    groupDes = data.groupby(['MatchID', 'DestinationPlayerID']).count()
    groupOri = pd.DataFrame(groupOri, columns=['TeamID'])
    groupDes = pd.DataFrame(groupDes, columns=['TeamID'])
    groupOri.rename(columns={'TeamID': 'count1'}, inplace=True)
    groupDes.rename(columns={'TeamID': 'count2'}, inplace=True)
    sumCount = pd.concat([groupOri, groupDes], axis=1)
    sumCount.fillna(0, inplace=True)
    sumCount['count'] = sumCount['count1'] + sumCount['count2']
    index = sumCount.index.to_frame(name=[0, 1])
    sumCount = pd.concat([sumCount['count'], index[0]], axis=1)  #每场比赛每名球员传球数（传出+接受）
    omega = sumCount.groupby([0]).sum()
    omega.rename(columns={'count': 'omega'}, inplace=True)
    mu = sumCount.groupby([0]).mean()
    mu.rename(columns={'count': 'mu'}, inplace=True)
    sigma = sumCount.groupby([0]).std()
    sigma.rename(columns={'count': 'sigma'}, inplace=True)

    #划分25个区域（5*5）
    data['ox'] = np.ceil(data['EventOrigin_x'] / 20)
    data['oy'] = np.ceil(data['EventOrigin_y'] / 20)
    data['dx'] = np.ceil(data['EventDestination_x'] / 20)
    data['dy'] = np.ceil(data['EventDestination_y'] / 20)

    #同理计算每场比赛每个区域传球数
    groupLo = data.groupby(['MatchID', 'ox', 'oy']).count()
    groupLd = data.groupby(['MatchID', 'dx', 'dy']).count()
    groupLo = pd.DataFrame(groupLo, columns=['TeamID'])
    groupLd = pd.DataFrame(groupLd, columns=['TeamID'])
    groupLo.rename(columns={'TeamID': 'count1'}, inplace=True)
    groupLd.rename(columns={'TeamID': 'count2'}, inplace=True)
    sumCountL = pd.concat([groupLo, groupLd], axis=1)
    sumCountL.fillna(0, inplace=True)
    sumCountL['count'] = sumCountL['count1'] + sumCountL['count2']
    index = sumCountL.index.to_frame(name=[0, 1, 2])
    sumCountL = pd.concat([sumCountL['count'], index[0]], axis=1)
    muL = sumCountL.groupby([0]).mean()
    muL.rename(columns={'count': 'muL'}, inplace=True)
    sigmaL = sumCountL.groupby([0]).std()
    sigmaL.rename(columns={'count': 'sigmaL'}, inplace=True)

    #划分时间（5分钟一组，加时不算）
    data['T'] = np.ceil(data['EventTime'] / 300)
    data = data.loc[data['T'] <= 9]

    #同理计算每场比赛每个区域传球数
    groupT = data.groupby(['MatchID', 'MatchPeriod', 'T']).count()
    groupT = pd.DataFrame(groupT, columns=['TeamID'])
    groupT.rename(columns={'TeamID': 'count'}, inplace=True)

    index = groupT.index.to_frame(name=[0, 1, 2])
    groupT = pd.concat([groupT['count'], index[0]], axis=1)
    muT = groupT.groupby([0]).mean()
    muT.rename(columns={'count': 'muT'}, inplace=True)
    sigmaT = groupT.groupby([0]).std()
    sigmaT.rename(columns={'count': 'sigmaT'}, inplace=True)

    #汇总计算H
    H = pd.concat([omega, mu, sigma, muL, sigmaL, muT, sigmaT], axis=1)
    H['H'] = 7 / (1 / H['omega'] + 1 / H['mu'] + 1 / H['sigma'] + 1 / H['muL'] + 1 / H['sigmaL'] + 1 / H['muT'] + 1 / H[
        'sigmaT'])

    return H


def countShot(df, Team):
    data = df.loc[df['TeamID'] == Team]
    data = data.loc[data['EventType'] == 'Shot']
    groupOri = data.groupby(['MatchID']).count()
    groupOri = pd.DataFrame(groupOri, columns=['TeamID'])
    groupOri.rename(columns={'TeamID': 'count1'}, inplace=True)
    return groupOri


def Duel(df, Team):
    End = pd.concat([df['End']], axis=1)
    End.fillna(method='bfill', inplace=True)
    Duel = pd.concat([df['MatchID'], df['Begin'], End['End']], axis=1)
    Duel.dropna(inplace=True)

    data = Duel.loc[Duel['Begin'] == Team]
    Begin = data.groupby(['MatchID']).count()
    data = data.loc[Duel['End'] == Team]
    End = data.groupby(['MatchID', 'End']).count()
    End.rename(columns={'Begin': 'End'}, inplace=True)
    index = End.index.to_frame(name=[0, 1])
    End.index = index[0]
    rate = pd.concat([Begin['Begin'], End['End']], axis=1)
    rate['rate'] = 1 - rate['End'] / rate['Begin']  #被Duel输比率

    return rate


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path = './Dataset/'
address_csv = 'passingevents.csv'
address_csv1 = 'matches.csv'
address_csv2 = 'fullevents.csv'
passing = pd.read_csv(path + address_csv)
full = pd.read_csv(path + address_csv2)
matches = pd.read_csv(path + address_csv1)

#计算H值

DuelHus = Duel(full, 'Huskies')
DuelOpp = Duel(full, 'Opponent1')
for i in range(2, 20):
    Team = 'Opponent' + str(i)
    DT = Duel(full, Team)
    DuelOpp = pd.concat([DuelOpp, DT], axis=0)
DuelOpp.rename(columns={'rate': 'rateOpp'}, inplace=True)
Duelcomp = pd.concat([DuelHus['rate'], DuelOpp['rateOpp']], axis=1)
Duelcomp.reset_index(drop=True, inplace=True)

H = Hscore(passing, 'Huskies')
HOpp = Hscore(passing, 'Opponent1')
for i in range(2, 20):
    Team = 'Opponent' + str(i)
    HT = Hscore(passing, Team)
    HOpp = pd.concat([HOpp, HT], axis=0)
HOpp.sort_index(inplace=True)
HOpp.rename(columns={'H': 'HOpp'}, inplace=True)
H.reset_index(drop=True, inplace=True)
HOpp.reset_index(drop=True, inplace=True)
HComp = pd.concat([H['H'], HOpp['HOpp'], matches['Outcome']], axis=1)
Htem = pd.DataFrame(columns=['Hhome', 'Haway'])
Point = pd.DataFrame(columns=['OwnPoint', 'OppPoint'])
HComh = pd.concat([H['H'], HOpp['HOpp'], matches['Side'], matches['Outcome'], Htem], axis=1)

for i in range(38):
    if HComp.loc[i, 'Outcome'] == 'win':
        HComp.loc[i, 'Outcome'] = 'r'
        Point.loc[i, 'OwnPoint'] = 3
        Point.loc[i, 'OppPoint'] = 0
    if HComp.loc[i, 'Outcome'] == 'tie':
        HComp.loc[i, 'Outcome'] = 'b'
        Point.loc[i, 'OwnPoint'] = 1
        Point.loc[i, 'OppPoint'] = 1
    if HComp.loc[i, 'Outcome'] == 'loss':
        HComp.loc[i, 'Outcome'] = 'k'
        Point.loc[i, 'OwnPoint'] = 0
        Point.loc[i, 'OppPoint'] = 3

for i in range(38):
    if HComh.loc[i, 'Side'] == 'away':
        HComh.loc[i, 'Hhome'] = HComh.loc[i, 'HOpp']
        HComh.loc[i, 'Haway'] = HComh.loc[i, 'H']
        if HComh.loc[i, 'Outcome'] == 'win':
            HComh.loc[i, 'Outcome'] = 'k'
        if HComh.loc[i, 'Outcome'] == 'tie':
            HComh.loc[i, 'Outcome'] = 'b'
        if HComh.loc[i, 'Outcome'] == 'loss':
            HComh.loc[i, 'Outcome'] = 'r'
    if HComh.loc[i, 'Side'] == 'home':
        HComh.loc[i, 'Hhome'] = HComh.loc[i, 'H']
        HComh.loc[i, 'Haway'] = HComh.loc[i, 'HOpp']
        if HComh.loc[i, 'Outcome'] == 'win':
            HComh.loc[i, 'Outcome'] = 'r'
        if HComh.loc[i, 'Outcome'] == 'tie':
            HComh.loc[i, 'Outcome'] = 'b'
        if HComh.loc[i, 'Outcome'] == 'loss':
            HComh.loc[i, 'Outcome'] = 'k'

#Hus与对手H值比较
plt.figure()
plt.title('Hus-Opp')
plt.xlabel('H_Hus')
plt.ylabel('H_Opp')
plt.scatter(HComp['H'], HComp['HOpp'], c=HComp['Outcome'], marker='o')
plt.plot([10, 60], [10, 60], linewidth=0.5)
plt.savefig('./Plots/Hus-Opp.png')
plt.show()

#Home、Away H值比较
plt.figure()
plt.title('Home-Away')
plt.xlabel('H_home')
plt.ylabel('H_away')
plt.scatter(HComh['Hhome'], HComh['Haway'], c=HComh['Outcome'], marker='o')
plt.plot([10, 60], [10, 60], linewidth=0.5)
plt.savefig('./Plots/Home-Away.png')
plt.show()

#Shot、Score、Point与H比较
Hshot = countShot(full, 'Huskies')
Oppshot = countShot(full, 'Opponent1')
for i in range(2, 20):
    Team = 'Opponent' + str(i)
    TS = countShot(full, Team)
    Oppshot = pd.concat([Oppshot, TS], axis=0)
Oppshot.sort_index(inplace=True)
Oppshot.rename(columns={'count1': 'Opp'}, inplace=True)
Hshot.rename(columns={'count1': 'Hus'}, inplace=True)
Hshot.reset_index(drop=True, inplace=True)
Oppshot.reset_index(drop=True, inplace=True)
HOpp.rename(columns={'omega': 'omegaOpp'}, inplace=True)
HOpp.rename(columns={'mu': 'muOpp'}, inplace=True)
HOpp.rename(columns={'sigma': 'sigmaOpp'}, inplace=True)
HOpp.rename(columns={'muL': 'muLOpp'}, inplace=True)
HOpp.rename(columns={'sigmaL': 'sigmaLOpp'}, inplace=True)
HOpp.rename(columns={'muT': 'muTOpp'}, inplace=True)
HOpp.rename(columns={'sigmaT': 'sigmaTOpp'}, inplace=True)
Shotcomp = pd.concat([Hshot['Hus'], Oppshot['Opp'], H, HOpp], axis=1)
Scorecomp = pd.concat([matches['OwnScore'], matches['OpponentScore'], H, HOpp], axis=1)
Pointcomp = pd.concat([Point, H, HOpp], axis=1)
Shotcorr = Shotcomp.corr(method='pearson')
Scorecorr = Scorecomp.corr(method='pearson')
Pointcorr = Pointcomp.corr(method='pearson')

#1 Pass-Shot
plt.figure()
plt.title('(a)')
plt.xlabel('Pass')
plt.ylabel('Shot')
p1 = plt.scatter(Shotcomp['omega'], Shotcomp['Hus'], c='r', marker='o')
p2 = plt.scatter(Shotcomp['omegaOpp'], Shotcomp['Opp'], c='b', marker='o')
plt.legend([p1, p2], ['Huskies', 'Opponent'])
plt.savefig('./Plots/Pass-Shot.png')
plt.show()
#2 Pass-Score
plt.figure()
plt.title('(b)')
plt.xlabel('Pass')
plt.ylabel('Score')
p1 = plt.scatter(Scorecomp['omega'], Scorecomp['OwnScore'], c='r', marker='o')
p2 = plt.scatter(Scorecomp['omegaOpp'], Scorecomp['OpponentScore'], c='b', marker='o')
plt.legend([p1, p2], ['Huskies', 'Opponent'])
plt.savefig('./Plots/Pass-Score.png')
plt.show()
#H-Shot
plt.figure()
plt.title('(a)')
plt.xlabel('H')
plt.ylabel('Shot')
p1 = plt.scatter(Shotcomp['H'], Shotcomp['Hus'], c='r', marker='o')
p2 = plt.scatter(Shotcomp['HOpp'], Shotcomp['Opp'], c='b', marker='o')
plt.legend([p1, p2], ['Huskies', 'Opponent'])
plt.savefig('./Plots/H-Shot.png')
plt.show()
#H-Score
plt.figure()
plt.title('(b)')
plt.xlabel('H')
plt.ylabel('Score')
p1 = plt.scatter(Scorecomp['H'], Scorecomp['OwnScore'], c='r', marker='o')
p2 = plt.scatter(Scorecomp['HOpp'], Scorecomp['OpponentScore'], c='b', marker='o')
plt.legend([p1, p2], ['Huskies', 'Opponent'])
plt.savefig('./Plots/H-Score.png')
plt.show()

KeyError: 'End'